# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe1bcc17fa0>
├── 'a' --> tensor([[ 2.3326,  1.6389,  2.0654],
│                   [-0.3836, -0.7664,  0.3492]])
└── 'x' --> <FastTreeValue 0x7fe1bcc17dc0>
    └── 'c' --> tensor([[-0.8663, -1.0136, -1.3421, -0.7492],
                        [ 0.0094,  1.2175, -0.5333, -0.3621],
                        [ 0.3999, -0.9843,  0.5236, -0.5714]])

In [4]:
t.a

tensor([[ 2.3326,  1.6389,  2.0654],
        [-0.3836, -0.7664,  0.3492]])

In [5]:
%timeit t.a

66.4 ns ± 0.0106 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe1bcc17fa0>
├── 'a' --> tensor([[-0.8449, -1.5767, -0.7022],
│                   [ 0.2507,  0.1502,  0.9692]])
└── 'x' --> <FastTreeValue 0x7fe1bcc17dc0>
    └── 'c' --> tensor([[-0.8663, -1.0136, -1.3421, -0.7492],
                        [ 0.0094,  1.2175, -0.5333, -0.3621],
                        [ 0.3999, -0.9843,  0.5236, -0.5714]])

In [7]:
%timeit t.a = new_value

71.1 ns ± 0.0366 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 2.3326,  1.6389,  2.0654],
               [-0.3836, -0.7664,  0.3492]]),
    x: Batch(
           c: tensor([[-0.8663, -1.0136, -1.3421, -0.7492],
                      [ 0.0094,  1.2175, -0.5333, -0.3621],
                      [ 0.3999, -0.9843,  0.5236, -0.5714]]),
       ),
)

In [10]:
b.a

tensor([[ 2.3326,  1.6389,  2.0654],
        [-0.3836, -0.7664,  0.3492]])

In [11]:
%timeit b.a

60.9 ns ± 0.0695 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.6645, -1.1029,  0.8463],
               [ 0.5755, -0.0129, -0.5367]]),
    x: Batch(
           c: tensor([[-0.8663, -1.0136, -1.3421, -0.7492],
                      [ 0.0094,  1.2175, -0.5333, -0.3621],
                      [ 0.3999, -0.9843,  0.5236, -0.5714]]),
       ),
)

In [13]:
%timeit b.a = new_value

496 ns ± 0.149 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

827 ns ± 0.648 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 13.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

141 µs ± 499 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 431 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe10eb2c310>
├── 'a' --> tensor([[[ 2.3326,  1.6389,  2.0654],
│                    [-0.3836, -0.7664,  0.3492]],
│           
│                   [[ 2.3326,  1.6389,  2.0654],
│                    [-0.3836, -0.7664,  0.3492]],
│           
│                   [[ 2.3326,  1.6389,  2.0654],
│                    [-0.3836, -0.7664,  0.3492]],
│           
│                   [[ 2.3326,  1.6389,  2.0654],
│                    [-0.3836, -0.7664,  0.3492]],
│           
│                   [[ 2.3326,  1.6389,  2.0654],
│                    [-0.3836, -0.7664,  0.3492]],
│           
│                   [[ 2.3326,  1.6389,  2.0654],
│                    [-0.3836, -0.7664,  0.3492]],
│           
│                   [[ 2.3326,  1.6389,  2.0654],
│                    [-0.3836, -0.7664,  0.3492]],
│           
│                   [[ 2.3326,  1.6389,  2.0654],
│                    [-0.3836, -0.7664,  0.3492]]])
└── 'x' --> <FastTreeValue 0x7fe107eb8f10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.9 µs ± 41.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe107e9ba30>
├── 'a' --> tensor([[ 2.3326,  1.6389,  2.0654],
│                   [-0.3836, -0.7664,  0.3492],
│                   [ 2.3326,  1.6389,  2.0654],
│                   [-0.3836, -0.7664,  0.3492],
│                   [ 2.3326,  1.6389,  2.0654],
│                   [-0.3836, -0.7664,  0.3492],
│                   [ 2.3326,  1.6389,  2.0654],
│                   [-0.3836, -0.7664,  0.3492],
│                   [ 2.3326,  1.6389,  2.0654],
│                   [-0.3836, -0.7664,  0.3492],
│                   [ 2.3326,  1.6389,  2.0654],
│                   [-0.3836, -0.7664,  0.3492],
│                   [ 2.3326,  1.6389,  2.0654],
│                   [-0.3836, -0.7664,  0.3492],
│                   [ 2.3326,  1.6389,  2.0654],
│                   [-0.3836, -0.7664,  0.3492]])
└── 'x' --> <FastTreeValue 0x7fe107e9b820>
    └── 'c' --> tensor([[-0.8663, -1.0136, -1.3421, -0.7492],
                        [ 0.0094,  1.2175, -0.5333, -0.3621],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 66.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.7 µs ± 459 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.8663, -1.0136, -1.3421, -0.7492],
                       [ 0.0094,  1.2175, -0.5333, -0.3621],
                       [ 0.3999, -0.9843,  0.5236, -0.5714]],
              
                      [[-0.8663, -1.0136, -1.3421, -0.7492],
                       [ 0.0094,  1.2175, -0.5333, -0.3621],
                       [ 0.3999, -0.9843,  0.5236, -0.5714]],
              
                      [[-0.8663, -1.0136, -1.3421, -0.7492],
                       [ 0.0094,  1.2175, -0.5333, -0.3621],
                       [ 0.3999, -0.9843,  0.5236, -0.5714]],
              
                      [[-0.8663, -1.0136, -1.3421, -0.7492],
                       [ 0.0094,  1.2175, -0.5333, -0.3621],
                       [ 0.3999, -0.9843,  0.5236, -0.5714]],
              
                      [[-0.8663, -1.0136, -1.3421, -0.7492],
                       [ 0.0094,  1.2175, -0.5333, -0.3621],
                       [ 0.3999, -0.9843,  0.5236, -0.5714]],

In [26]:
%timeit Batch.stack(batches)

77.5 µs ± 223 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.8663, -1.0136, -1.3421, -0.7492],
                      [ 0.0094,  1.2175, -0.5333, -0.3621],
                      [ 0.3999, -0.9843,  0.5236, -0.5714],
                      [-0.8663, -1.0136, -1.3421, -0.7492],
                      [ 0.0094,  1.2175, -0.5333, -0.3621],
                      [ 0.3999, -0.9843,  0.5236, -0.5714],
                      [-0.8663, -1.0136, -1.3421, -0.7492],
                      [ 0.0094,  1.2175, -0.5333, -0.3621],
                      [ 0.3999, -0.9843,  0.5236, -0.5714],
                      [-0.8663, -1.0136, -1.3421, -0.7492],
                      [ 0.0094,  1.2175, -0.5333, -0.3621],
                      [ 0.3999, -0.9843,  0.5236, -0.5714],
                      [-0.8663, -1.0136, -1.3421, -0.7492],
                      [ 0.0094,  1.2175, -0.5333, -0.3621],
                      [ 0.3999, -0.9843,  0.5236, -0.5714],
                      [-0.8663, -1.0136, -1.3421, -0.7492],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 204 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

317 µs ± 873 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
